# 🏦 Sequential Workflow with Custom Compliance Executor

## Overview

This notebook demonstrates mixing AI agents with **custom executors** in a financial services workflow. We'll create a loan advisory pipeline where:

1. **Financial Advisor Agent** - Provides personalized loan product recommendations
2. **Compliance Executor** - Custom code that adds required regulatory disclosures

### 💼 Industry Use Case: Loan Product Recommendations with Compliance

A customer asks about loan products. The workflow:
1. AI agent provides helpful product recommendations
2. Custom executor ensures all responses include required compliance disclaimers

### ⚠️ Important Financial Disclaimer
> **This notebook is for educational and demonstration purposes only.** Always consult licensed financial professionals for actual financial advice.

### Key Concepts

| Concept | Description |
|---------|-------------|
| **Custom Executor** | Python class that processes conversation programmatically |
| **Handler Contract** | Accept `list[ChatMessage]`, emit updated list via `ctx.yield_output()` |
| **Mixed Pipeline** | Combine AI agents with deterministic code processors |

### Architecture

```
Customer Question
    ↓
[input-conversation]
    ↓
Financial Advisor Agent (recommendations)
    ↓
[to-conversation:advisor]
    ↓
Compliance Executor (adds disclaimers)
    ↓
[complete]
    ↓
Compliant Response with Disclaimers
```

## Prerequisites

- ✅ Azure OpenAI Service configured
- ✅ Environment variables: `AZURE_OPENAI_ENDPOINT`, `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- ✅ Azure CLI authentication: Run `az login`

## 1️⃣ Setup and Imports

In [ ]:
import asyncio
from typing import Any

import os
from dotenv import load_dotenv
from azure.identity import AzureCliCredential

from agent_framework import (
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    Role,
    SequentialBuilder,
    WorkflowContext,
    handler,
)
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables from .env file
load_dotenv('../../.env')

# Verify environment is loaded
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
print(f"✅ Environment loaded: {azure_endpoint is not None and deployment_name is not None}")

## 2️⃣ Create Custom Compliance Executor

This custom executor adds required regulatory disclaimers to all financial advice responses. In real banking applications, this ensures:

- **Regulatory Compliance**: All customer communications include required disclosures
- **Consistency**: Same disclaimers added regardless of which agent responds
- **Audit Trail**: Programmatic guarantee that compliance was applied

### Handler Contract
```python
@handler
async def handler_name(self, conversation: list[ChatMessage], ctx: WorkflowContext) -> None:
    # Process conversation
    # Add compliance content
    await ctx.yield_output(updated_conversation)
```

In [ ]:
class ComplianceExecutor(Executor):
    """Custom executor that adds regulatory compliance disclaimers to financial advice.
    
    Note: When following an agent in a sequential workflow, the input is AgentExecutorResponse
    (not list[ChatMessage]). We extract full_conversation from the response.
    """

    COMPLIANCE_DISCLAIMER = """
---
📋 **IMPORTANT DISCLOSURES**

⚠️ This information is for educational purposes only and does not constitute financial advice.
• Loan approval subject to credit approval and verification of information
• APR and terms may vary based on creditworthiness and other factors
• Past performance does not guarantee future results
• Please consult with a licensed financial advisor for personalized advice
• FDIC insured where applicable | Equal Housing Lender

*Your institution name here* | NMLS# 123456
---
"""

    @handler
    async def add_compliance(
        self, 
        agent_response: AgentExecutorResponse, 
        ctx: WorkflowContext[list[ChatMessage]]
    ) -> None:
        """Process agent response and add compliance disclaimers.
        
        Args:
            agent_response: Response from the previous agent (AgentExecutorResponse)
            ctx: Workflow context for sending output
        """
        # Extract conversation from agent response
        conversation = agent_response.full_conversation
        
        if not conversation:
            await ctx.send_message([ChatMessage(role=Role.ASSISTANT, text="No conversation to process.")])
            return
        
        # Count messages for audit logging
        user_msgs = sum(1 for m in conversation if m.role == Role.USER)
        assistant_msgs = sum(1 for m in conversation if m.role == Role.ASSISTANT)
        
        print(f"📋 Compliance Executor: Processing {user_msgs} user and {assistant_msgs} assistant messages")
        
        # Add compliance disclaimer as a final message
        compliance_message = ChatMessage(
            role=Role.ASSISTANT,
            text=self.COMPLIANCE_DISCLAIMER
        )
        
        final_conversation = list(conversation) + [compliance_message]
        print("✅ Compliance disclaimers added to response")
        
        await ctx.send_message(final_conversation)

## 3️⃣ Build and Execute Loan Advisory Workflow

### Participant Order
1. **Financial Advisor Agent**: AI-powered loan product recommendations
2. **Compliance Executor**: Deterministic compliance disclaimer insertion

In [ ]:
async def run_loan_advisory_workflow() -> None:
    # Create Azure OpenAI chat client
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )
    print("✅ Azure OpenAI Chat Client created")
    
    # Financial Advisor Agent - Provides loan product recommendations
    financial_advisor = chat_client.as_agent(
        instructions=(
            "You are a Financial Advisor at a retail bank. Help customers understand their loan options.\n"
            "When discussing loans:\n"
            "1. Explain available products (personal loans, auto loans, mortgages, HELOCs)\n"
            "2. Discuss general eligibility factors (credit score, income, debt-to-income ratio)\n"
            "3. Mention typical rate ranges (without making specific promises)\n"
            "4. Be helpful but never make guarantees about approval or specific rates\n"
            "Keep responses concise and educational."
        ),
        name="financial_advisor",
    )
    print("✅ Financial Advisor Agent created")

    # Build sequential workflow: financial_advisor -> compliance_executor
    compliance_executor = ComplianceExecutor(id="compliance")
    workflow = SequentialBuilder().participants([financial_advisor, compliance_executor]).build()
    print("✅ Workflow built: Financial Advisor → Compliance Executor")

    # Customer question about loans
    customer_question = "I'm looking to buy a car and need about $30,000. What are my loan options and what factors will affect my rate?"

    print("\n" + "=" * 60)
    print("💬 CUSTOMER INQUIRY")
    print("=" * 60)
    print(f"👤 Customer: {customer_question}")
    print("=" * 60)

    # Run and print final conversation
    events = await workflow.run(customer_question)
    outputs = events.get_outputs()

    if outputs:
        print("\n📄 RESPONSE WITH COMPLIANCE")
        print("=" * 60)
        messages: list[ChatMessage] | Any = outputs[0]
        for i, msg in enumerate(messages, start=1):
            name = msg.author_name or ("assistant" if msg.role == Role.ASSISTANT else "user")
            role_emoji = "👤" if msg.role == Role.USER else "🏦"
            print(f"\n{'-' * 60}")
            print(f"{role_emoji} [{name.upper()}]")
            print(f"{'-' * 60}")
            print(f"{msg.text}")
    
    print("\n" + "=" * 60)
    print("✅ Loan advisory workflow complete!")

## 4️⃣ Run the Loan Advisory Workflow

In [ ]:
await run_loan_advisory_workflow()

## 📝 Key Takeaways

### Why Custom Executors for FSI?

| Benefit | Description |
|---------|-------------|
| **Guaranteed Compliance** | Code-based disclaimers can't be "forgotten" by AI |
| **Audit Trail** | Deterministic processing for regulatory audits |
| **Separation of Concerns** | AI handles advice, code handles compliance |
| **Customizable** | Easily update disclaimers without retraining models |

### Industry Use Cases for Custom Executors

```python
# Compliance Executor - Add required disclosures
# PII Redactor - Remove sensitive data before logging
# Rate Limiter - Throttle customer interactions
# Audit Logger - Record all interactions for compliance
# Sentiment Checker - Flag negative interactions for review
```

### Production Considerations

- Custom executors should be **lightweight and focused**
- Add **error handling** for edge cases
- Consider **timeout handling** for complex processing
- **Test independently** before integration
- **Log processing** for debugging and compliance